# Predicting reports of bullying, racism, and unwanted sexual behavior from app store reviews

In the [Washington Post's analysis](https://www.washingtonpost.com/technology/2019/11/22/apple-says-its-app-store-is-safe-trusted-place-we-found-reports-unwanted-sexual-behavior-six-apps-some-targeting-minors/), they designed a machine learning algorithm to detect patterns of "unsafe" behavior through analyzing App Store reviews. Using a spreadsheet of hand-tagged reviews, we'll both train an algorithm to spot these behaviors as well as learn what words tips our algorithm off.

<p class="reading-options">
  <a class="btn" href="/wapo-app-reviews/predict-reviews">
    <i class="fa fa-sm fa-book"></i>
    Read online
  </a>
  <a class="btn" href="/wapo-app-reviews/notebooks/Predict reviews.ipynb">
    <i class="fa fa-sm fa-download"></i>
    Download notebook
  </a>
  <a class="btn" href="https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/wapo-app-reviews/notebooks/Predict reviews.ipynb" target="_new">
    <i class="fa fa-sm fa-laptop"></i>
    Interactive version
  </a>
</p>

### Prep work: Downloading necessary files
Before we get started, we need to download all of the data we'll be using.
* **reviews-marked.csv:** combined app reviews - US-based reviews of multiple "random chat apps" from Apple's App Store (Chat for Strangers, Holla, Skout, and Yubo)
* **reviews-marked.csv:** annotated reviews - Reviews of "random chat apps," marked for racist, bullying, or sexual behavior


In [ ]:
# Make data directory if it doesn't exist
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/wapo-app-reviews/data/reviews-marked.csv -P data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/wapo-app-reviews/data/reviews-marked.csv -P data

In [3]:
import pandas as pd
pd.set_option("display.max_colwidth", 300)

In [4]:
df = pd.read_csv("data/reviews-marked.csv")
df = df.dropna(subset=['Review'])
df.shape

(56055, 9)

In [5]:
df.head()

,Country,Date,Rating,Review,Version,source,racism,bullying,sexual
0,US,11/22/2019,5,It’s a great app to meet new people and chat in very satisfied with downloading this app i recommend this app if you like to chat or just to meet new people. And you can choose which country To find different users!,4.4.5,holla,NaN,NaN,NaN
1,US,11/22/2019,5,"Holla is an excellent app, where I get to know new people every time and even get to make new friends. I truly recommend this application to all people!",4.4.5,holla,NaN,NaN,NaN
2,US,11/22/2019,1,Get rid of micro transactions or i will find a new app to use. Why should i have to pay for that it’s so stupid,3.8,holla,0.0,0.0,0.0
3,US,11/22/2019,5,"Free to use app, meet people around the world.",-,holla,NaN,NaN,NaN
4,US,11/21/2019,5,I got this app and everything has been different. I’ve met so many interesting people. From around the world. I was recently reunited with my high school girlfriend. We’re getting married. I met and married The love of my Life thanks to Holla. Thanks Holla!!!!!,4.4.5,holla,NaN,NaN,NaN


We've only filled in `0` and `1` for racism, bullying and unwanted sexual behavior in a handful of reviews, so we'll filter them into known and unknown. We'll use the known ones to train our classifier, and then run it on unknown to find possible reviews to examine.

In [6]:
known = df[df.sexual.notna()].copy()
unknown = df[df.sexual.isna()].copy()

In [7]:
known.shape

(330, 9)

## Vectorize our text

In [8]:
%%time

from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

stemmer = SnowballStemmer('english')
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer,self).build_analyzer()
        return lambda doc:(stemmer.stem(word) for word in analyzer(doc))

vectorizer = StemmedTfidfVectorizer()

X = vectorizer.fit_transform(known.Review)
words_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
words_df.head(5)

CPU times: user 197 ms, sys: 6.6 ms, total: 203 ms
Wall time: 216 ms


,000,10,100,13,14,15,16,17,18,19,...,للدردشة,للرمنسية,للعب,مخصص,مكان,من,نطاق,والصداقة,وضع,ومكان
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.116557,0.0,0.106118,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


What's the split between "normal" reviews and ones featuring unwanted sexual behavior?

In [9]:
known.sexual.value_counts()

0.0    314
1.0     16
Name: sexual, dtype: int64

## Train a classifier

In [21]:
%%time

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

X = vectorizer.fit_transform(known.Review)
#X = words_df
y = known.sexual

#clf = SVC(kernel='linear', probability=True, class_weight='balanced')
#clf = LogisticRegression(solver='lbfgs', class_weight='balanced')
#clf = MultinomialNB()
#clf = RandomForestClassifier(n_estimators=100)
clf = LinearSVC(class_weight='balanced')
clf.fit(X, y)
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler()
# X_resampled, y_resampled = ros.fit_resample(X, y)
# clf.fit(X_resampled, y_resampled)

CPU times: user 191 ms, sys: 3.8 ms, total: 195 ms
Wall time: 196 ms


LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

Last time with our vectorizer we used `.fit_transform`, which taught the vectorizer all of the words in the reviews and then transformed them into numbers the algorithm could understand. This time we just use `.transform` because **any words not in our original dataset won't actually have meaning**. The algo won't have seen them before, so how could it know what category they might imply?

In [23]:
X = vectorizer.transform(unknown.Review)

unknown['predicted'] = clf.predict(X)
unknown['predicted_proba'] = clf.decision_function(X)
# Depending on the classifier, you use .decision_function or .predict_proba
#unknown['predicted_proba'] = clf.predict_proba(X)[:, 1]

In [26]:
unknown.predicted.value_counts()

0.0    55428
1.0      297
Name: predicted, dtype: int64

While there are 297 that have been predicted to probably be about unwanted sexual behavior, we might want to look at more than that!

For a LinearSVC, the closer to `0` the `.decision_function`, the more uncertain the classifier is. High positive values mean it's sure the review is unwanted sexual behavior, high negative values means it's sure a review isn't.

Instead of just taking reviews the classifier marked as definitely interesting to us, we could also sort by `.decision_function` and take the top 500 or 1000, just to be sure to really check those edge cases.

In [28]:
unknown.sort_values(by='predicted_proba', ascending=False).head(20)

,Country,Date,Rating,Review,Version,source,racism,bullying,sexual,predicted,predicted_proba
19428,US,11/17/2018,2,All the guys on here ever ask for is nudes like I don't want to send my nudes to you,4.12.1,skout,NaN,NaN,NaN,1.0,1.002086
11002,US,06/23/2019,1,To many perverts and all they ask for is nudes🙄,3.9.3,chat-for-strangers,NaN,NaN,NaN,1.0,0.974331
53538,US,07/24/2015,3,I like this app but there is so many horny guys and they are all 30 and asking for nudes,3.5,chat-for-strangers,NaN,NaN,NaN,1.0,0.791301
55612,US,02/23/2012,4,Like a small thing that's pink/ blue to show whether your m or f. That would be nice. And to all the guys out there... Put your dick back in your pants. I'm a guy but I don't creep on girls. There a thing called porn.,1.4,chat-for-strangers,NaN,NaN,NaN,1.0,0.762625
17338,US,04/15/2019,1,Basically men/women asking for nudes all I wanted to do is chat with my cousin,2.85,chat-for-strangers,NaN,NaN,NaN,1.0,0.699913
53557,US,07/10/2015,1,All the guys on skout are so ugly omg and half of them are fake. Old man posing as 20 year olds. Perverts everywhere. Scammers talking about do you want to make 1500 a month! SMH! This whole app is a joke. Would not recommend if you are looking for a relationship....not even a friendship. If you...,5.8.2,skout,NaN,NaN,NaN,1.0,0.683701
12030,US,11/04/2018,4,This app is disgusting. If you look hard enough you will find underaged girls selling their nude photos. And even worse girls are sending nudes to guys over 18 with just a few questions asked,3.7.2,holla,NaN,NaN,NaN,1.0,0.662834
55129,US,07/03/2012,3,"All there is, is 20 year old men that are perverted and are horny and always wants girls kiks... Well I'm an 11 year old girl and one guy was nice in a minute, and when he came on my kik he was nasty😝!! Sooo what I'm trying to say is... PLZ BLOCK THE PERVERTS!!",1.5,chat-for-strangers,NaN,NaN,NaN,1.0,0.660950
50262,US,02/18/2013,5,Nude nude and more nude,3.3.1,skout,NaN,NaN,NaN,1.0,0.659748
7849,US,03/27/2017,5,All guys,1,holla,NaN,NaN,NaN,1.0,0.606504


While we looked at them in the notebook here, it's probably better to save them to a CSV and take a better look later. We'll take the **top 500 most likely** for later research.

In [34]:
to_investigate = unknown.sort_values(by='predicted_proba', ascending=False).head(500)
to_investigate.to_csv("data/to-investigate.csv", index=False)

### Explaining our classifier

While we're at it, what's the classifier even doing? We can assume that it's looking at the words and figuring out that certain words signal unwanted sexual behavior or not, but which words do which things?

This is called **explainability**, and it's a big part of being able to argue about whether your machine learning algorithm makes sense or not. The specific question of which words are more or less important is called **feature importance**.

There's a lot of different, somewhat complicated ways to ask scikit-learn what features are important, but there's a Python library called [eli5](https://eli5.readthedocs.io/) that does an almost-perfect job making it only take one line of code.

In [35]:
import eli5

# eli5 gets our classifier and our vectorizer, so it knows what
# numbers are what words (otherwise you just get numbers and weights)
eli5.explain_weights(clf, vec=vectorizer)

Weight?,Feature
+1.684,nude
+1.463,guy
+1.186,pervert
+1.147,men
+1.083,jack
+1.060,find
+1.045,thing
+0.980,dick
+0.868,off
+0.833,on


Seems reasonable.

## Review

We have a lot of app store reviews, some of which we have **manually tagged** for certain kinds of behavior. We use those to **train** a machine learning algorithm, teaching it which words are associated with which kinds of behavior.

After we've trained the algorithm on our **known, labeled reviews** we can then send it **unknown reviews**, and it will flag those we need to review. Since we don't mind reviewing a large number of reviews, we also save a handful that the algorithm flagged as barely not-interesting.

## Discussion topics

Did we label enough reviews?

Beyond the idea of "more data is better," why might we want to label more reviews? Think about using `.fit_transform` vs just `.transform`.

If we need to label more reviews, would it be helpful to search for words like "nude" and "guy" to see if we can easily track down some unwanted sexual behavior? Or should we just keep going one-by-one?

Why did we use a **stemmer?** If we had thousands of labeled reviews instead of just a few dozen, would that have been as necessary?